In [1]:
import numpy as np

In [2]:
pima = np.genfromtxt('heart.dat', delimiter=' ')

In [3]:
def sghmc(Y, X, stogradU, M, eps, m, theta, C, V):
    n = X.shape[0]
    p = X.shape[1]
    
    # Randomly sample momentum
    r = np.random.multivariate_normal(np.zeros(M.shape[0]),M)[:,np.newaxis]
    
    # Precompute
    B = 0.5 * V * eps
    D = 2*(C-B)*eps
    Minv = np.linalg.inv(M)
    
    # Hamiltonian dynamics
    for i in range(m):
        theta = theta + (eps*np.linalg.inv(M) @ r).ravel()
        r = r - eps*stogradU(theta, Y, X, nbatch) - eps*C @ Minv @ r \
            + np.random.multivariate_normal(np.zeros(M.shape[0]),D)[:,np.newaxis]

    return(theta)

In [4]:
def logistic(x):
    return 1/(1+np.exp(-x))

def stogradU(theta, Y, X, nbatch):
    '''A function that returns the stochastic gradient. Adapted from Eq. 5.
    Inputs are:
        theta, the parameters
        Y, the response
        X, the covariates
        nbatch, the number of samples to take from the full data
    '''
    alpha=5
    n = X.shape[0]
    batch_id = np.random.choice(np.arange(n),nbatch,replace=False)
    
    Y_pred = logistic(X[batch_id,:] @ theta[:,np.newaxis])
    epsilon = (Y[batch_id][:,np.newaxis] - Y_pred)
    
    grad = -n/nbatch * X[batch_id,:].T @ epsilon - theta[:,np.newaxis]/alpha
    
    return grad

In [5]:
X = np.concatenate((np.ones((pima.shape[0],1)),pima[:,0:pima.shape[1]]), axis=1)
Y = pima[:,pima.shape[1]-1]-1

Xs = (X - np.mean(X, axis=0))/np.concatenate((np.ones(1),np.std(X[:,1:], axis=0)))
n, p = X.shape

nsample = 10
nbatch = 270
M = np.identity(p)
C = 0 * np.identity(p)
eps = 0.1
m = 50
V = 0 * np.identity(p)
theta = np.zeros(p)

In [6]:
np.random.seed(2)

# w noise, no mh
samples = np.zeros((nsample, p))

for i in range(nsample):
    theta = sghmc(Y, Xs, stogradU, M, eps, m, theta, C, V)
    samples[i] = theta

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: overflow encountered in exp
  from ipykernel import kernelapp as app


In [7]:
np.mean(samples, 0)

array([  -770657.47287623,   -194349.31221372,   3202298.98557206,
         5155286.56804377,   1705675.21479073,   3161494.86855035,
          680696.61957205,   3427506.31725963,  -1871077.85411747,
         2345329.89054276,   3576742.20638365,    547479.23258703,
         4862971.24714529,   3278527.88515207,  25933515.41778655])

In [8]:
np.std(Xs,axis=0)

array([ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.])

In [9]:
np.mean(Xs,axis=0)

array([  0.00000000e+00,   3.45402719e-16,  -6.33238318e-17,
         4.68760833e-17,   5.72381648e-16,  -2.30679673e-16,
         6.20902506e-17,  -3.45402719e-17,  -9.62193288e-17,
         7.31924809e-17,   3.56916143e-16,  -1.43917799e-16,
        -2.30268479e-17,   1.31170794e-16,   8.96402294e-17])